In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import psycopg2

In [5]:
!pip install redis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 8.8 MB/s eta 0:00:00


In [6]:
import redis

In [ ]:
r = redis.Redis(
  host='redis-17099.c8.us-east-1-2.ec2.cloud.redislabs.com',
  port=17099,
  password='xGOqKDKeqC1Sd63n6omhWyGyTFx4syQC')

In [ ]:
# Postgres Database Information
pg_host = '35.237.226.12'
pg_database = 'telecommunications_data'
pg_port = 5432
pg_user = 'postgres'
pg_password = 'password'

In [ ]:
# Redis Client Object
redis_client = redis.Redis(host=redis_host, port=redis_port, password=redis_password)

In [7]:
def extract_data():

    # Extract data from CSV file using pandas
        df = pd.read_csv('customer_call_logs.csv')
        print('Extraction Successful')
        return df

In [8]:
#Let us define the transfrom_data() fucntion
def transform_data():
    """
    Retrieve data from Redis cache and transform (clean, structure, format)
    """
    try:
        # Retrieve data from Redis cache
        data = pd.read_json(redis_client.get('customer_call_logs').decode('utf-8'))

        # Transform data (clean, structure, format)
        data['call_cost_usd'] = data['call_cost'].apply(lambda x: float(x[1:]))
        data['call_date'] = pd.to_datetime(data['call_date'])
        data['call_duration_min'] = data['call_duration'].apply(lambda x: float(x.split(':')[0]) + float(x.split(':')[1])/60)
        transformed_data = data[['customer_id', 'call_cost_usd', 'call_destination', 'call_date', 'call_duration_min']]

        print('Data transformation successful.')
        return transformed_data
    
    except Exception as e:
        print('An error occurred while transforming data:', e)

In [ ]:
#Let us define the load_data() fucntion
def load_data(transformed_data):
    """
    Connect to Postgres database and load transformed data
    """
    try:
        # Connect to Postgres database
        conn = psycopg2.connect(host=pg_host, database=pg_database, user=pg_user, password=pg_password)

        # Create a cursor object
        cur = conn.cursor()

        # Create a table to store the data
        cur.execute('CREATE TABLE IF NOT EXISTS customer_call_logs (\
                     customer_id INT,\
                     call_cost_usd FLOAT,\
                     call_destination VARCHAR,\
                     call_date TIMESTAMP,\
                     call_duration_min FLOAT\
                     )')

        # Insert the transformed data into the database
        for i, row in transformed_data.iterrows():
            cur.execute(f"INSERT INTO customer_call_logs (customer_id, call_cost_usd, call_destination, call_date, call_duration_min) VALUES ({row['customer_id']}, {row['call_cost_usd']}, '{row['call_destination']}', '{row['call_date']}', {row['call_duration_min']})")

        # Commit the changes
        conn.commit()

        # Close the cursor and connection
        cur.close()
        conn.close()

        print('Data loading successful.')

In [ ]:
#Defining the main function

def data_pipeline():
    """
    Data pipeline function that extracts, transforms, and loads data
    """
    extract_data()
    transformed_data = transform_data()
    load_data(transformed_data)

if __name__ == '__main__':
    # Run the data pipeline function
    data_pipeline()